## Imports

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
)

colors = pc.qualitative.D3
angstrom = "\u212B"
Theta = "\u0398"
phi = "\u03C6"

# Load Data

In [ ]:
# ## Convert gold corrected data ##

# ddir = r"E:\atully\arpes_data\2023_February"

# # STEP 1 ##
# # Convert ibw to hdf5
# fn = "Au_000_g.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, theta, energy = HDF5_loader.load_hdf5(ddir, "Au_000_g.h5")  # load data from hdf5
# data.shape, theta.shape, energy.shape

In [ ]:
## 6 eV ##

ddir = r"E:\atully\arpes_data\2023_February"

## evaporated Au reference scan for EF correction and determining energy resolution
file = "Au_000_g.h5"  # PE=10, slit=400, sample temp = 32 K, EF = 1.91
# file = "Au_005_g.ibw"  # PE=10, slit=700, sample temp = 32 K, EF = 1.94

xlim = (-15, 15)
ylim = (1.85, 1.97)

In [ ]:
## XUV TR damaged sample measurement ##

ddir = r"E:\atully\arpes_data\2022_April\ARPES\Shield"

file = "FE2_H19_ang_nofilter.h5"
# file = "FE2_H19_tr_nofilter.h5"

xlim = (-15, 15)
ylim = (18.4, 18.6)

In [ ]:
data, theta, energy = HDF5_loader.load_hdf5(ddir, file)  # load data from hdf5

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    title=f"{file}",
    xaxis_title=f"{Theta}",
    yaxis_title="Energy (eV)",
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(
        x=theta, y=energy, z=analysis_functions.norm_data(data), coloraxis="coloraxis"
    )
)

fig.show()

In [ ]:
## FFT + Limit Data ##

data, theta, energy = HDF5_loader.load_hdf5(ddir, file)  # load data from hdf5

f_data = filter_functions.fft2d_mask(data, plot=False)

x_plot, y_plot, d_plot = analysis_functions.limit_dataset(
    # theta, energy, f_data, xlim=(-15, 15), ylim=(1.7, 2.3)
    theta,
    energy,
    f_data,
    xlim=xlim,
    ylim=ylim,
)

d_plot = analysis_functions.norm_data(d_plot)

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    # title=f"{file}",
    title=f"Au Reference Sample",
    xaxis_title=f"{Theta}",
    yaxis_title="Energy (eV)",
    equiv_axes=False,
    height=600,
    width=600,
)

fig.add_trace(go.Heatmap(x=x_plot, y=y_plot, z=d_plot, coloraxis="coloraxis"))

fig.show()

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    title=f"EDC of Au Reference Sample",
    xaxis_title="Energy (eV)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_plot, y=y_plot, data=d_plot, ylims=None, x_range=None
)

# Plot Data
fig.add_trace(go.Scatter(x=y_1d, y=col, name="data", line=dict(color=colors[0])))

# ## For After Fit
# fig.add_trace(
#     go.Scatter(
#         x=x, y=fit.eval(x=y_1d), name="FDconvGauss fit", line=dict(color=colors[1])
#     )
# )

# components = fit.eval_components(x=y_1d)
# for model_name, model_value in list(components.items())[0:1]:
#     fig.add_annotation(
#         # x=fit.params[f"center"].value,
#         # y=fit.eval(x=fit.params[f"{model_name}center"].value),
#         xref="x domain",
#         yref="y domain",
#         x=0.95,
#         y=0.9,
#         showarrow=False,
#         text=f'Energy Res: {tr_functions.sig_to_fwhm(fit.params["iA__sigma"].value)*1000:.1f} +/- {tr_functions.sig_to_fwhm(fit.params["iA__sigma"].stderr)*1000:.1f} meV',
#         font=dict(size=18),
#     )

fig.show()

In [ ]:
## Fit using convolved FD + Gaussian ##
import lmfit as lm

# T = 32  # measurement temp
T = 20  # measurement temp
k_B = 8.617333e-5  # eV/K

x = y_1d
data = col

offset_type = "constant"


## FD
def fermi_dirac(x, center, theta, amp):
    arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
    return -amp / 2 * np.tanh(arg)


## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)


## Gaussian
def gaussian(x, mu, sigma, amplitude):
    return (
        amplitude
        / (sigma * np.sqrt(2 * np.pi))
        * np.exp(-((x - mu) ** 2) / (2 * sigma**2))
    )


# ## Convolved model
# def conv_FDgauss(
#     x, center, theta, amp, const, fd_lin, iA__center, iA__sigma, iA__amplitude
# ):
#     FD = fermi_dirac(x, center, theta, amp)
#     # linear = (fd_lin * x) * (FD - np.min(FD))
#     # FD += const + linear
#     FD += const
#     gauss_x = np.arange(-3 * iA__sigma, 3 * iA__sigma, x[1] - x[0])
#     gauss1 = gaussian(gauss_x, iA__center, iA__sigma, iA__amplitude)
#     if len(gauss_x) > len(x):
#         print("gauss_x is longer than x array for FD")
#     left_len = int(len(gauss_x) / 2)
#     right_len = len(gauss_x) - left_len - 1
#     FD = np.concatenate(
#         (np.ones(left_len) * FD[0], FD, np.ones(right_len) * FD[-1]),
#         axis=0,
#         dtype=np.float32,
#     )  # adds the repeated values on left and right side; valid region is the same but edge effects aren't as bad
#     # because you're assuming something similar on the edges
#     conv = np.convolve(gauss1, FD, mode="valid").astype(np.float32)
#     # conv[: int(len(gauss_x) / 2)] = np.nan
#     # conv[-int(len(gauss_x) / 2) :] = np.nan
#     return conv


## Convolved model
def conv_FDgauss(x, center, theta, amp, const, iA__center, iA__sigma, iA__amplitude):
    FD = fermi_dirac(x, center, theta, amp)
    FD += const
    gauss_x = np.arange(-3 * iA__sigma, 3 * iA__sigma, x[1] - x[0])
    gauss1 = gaussian(gauss_x, iA__center, iA__sigma, iA__amplitude)
    if len(gauss_x) > len(x):
        print("gauss_x is longer than x array for FD")
    left_len = int(len(gauss_x) / 2)
    right_len = len(gauss_x) - left_len - 1
    FD = np.concatenate(
        (np.ones(left_len) * FD[0], FD, np.ones(right_len) * FD[-1]),
        axis=0,
        dtype=np.float32,
    )  # adds the repeated values on left and right side; valid region is the same but edge effects aren't as bad
    # because you're assuming something similar on the edges
    conv = np.convolve(gauss1, FD, mode="valid").astype(np.float32)

    # conv = np.convolve(gauss1, FD, mode="same").astype(np.float32)
    # conv[: int(len(gauss_x) / 2)] = np.nan
    # conv[-int(len(gauss_x) / 2) :] = np.nan
    return conv


full_model = lm.models.Model(conv_FDgauss)

params = full_model.make_params()

# params["center"].value = 1.9
params["center"].value = 18.48
params["center"].vary = True
# params["center"].min = 1.85
# params["center"].max = 1.95

params["theta"].value = k_B * T
params["theta"].vary = False
# params["theta"].vary = True

params["amp"].value = 3.667e-4
params["amp"].vary = True
params["amp"].min = 1e-8

params["const"].value = 0
# params["fd_lin"].value = 10
# params["fd_const"].value = 0

params["iA__center"].value = 0
params["iA__center"].vary = False


# ## 6 eV ##
# params["iA__sigma"].value = tr_functions.fwhm_to_sig(
#     0.02
# )  # 80 meV energy resolution for 6 eV laser
# params["iA__sigma"].vary = True
# params["iA__sigma"].min = tr_functions.fwhm_to_sig(0.01)
# params["iA__sigma"].max = tr_functions.fwhm_to_sig(0.1)


## XUV ##
params["iA__sigma"].value = tr_functions.fwhm_to_sig(
    0.08
)  # 50 - 100 meV energy resolution for 6 eV laser
params["iA__sigma"].vary = True
params["iA__sigma"].min = tr_functions.fwhm_to_sig(0.01)
params["iA__sigma"].max = tr_functions.fwhm_to_sig(0.2)


params["iA__amplitude"].value = 1
params["iA__amplitude"].vary = False

for param in params:
    params[param].value = np.float32(params[param].value)
    params[param].min = np.float32(params[param].min)
    params[param].max = np.float32(params[param].max)
    # print(params[param].value.dtype)


# fit = full_model.fit(data, x=x, params=params, nan_policy="propagate")
fit = full_model.fit(data.astype(np.float32), x=x.astype(np.float32), params=params)
# if plot_fit:
fit.plot()

# linear_params = fit.params.copy()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=x, y=fit.eval(x=x), name="fit"))
# fig.add_trace(go.Scatter(x=x, y=fit.init_fit, name="init"))
# fig.show()

In [ ]:
## Plot Fit Components ##

fig = tr_functions.thesis_fig(
    title=f"Fit Components",
    xaxis_title="Energy (eV)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=900,
)

gauss_x = np.arange(
    -3 * fit.params["iA__sigma"], 3 * fit.params["iA__sigma"], y_1d[1] - y_1d[0]
)

components = fit.eval_components(x=y_1d)
for model_name, model_value in components.items():
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=model_value,
            name=model_name,
        )
    )

    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=gaussian(
                x=gauss_x,
                mu=fit.params["iA__center"],
                sigma=fit.params["iA__sigma"],
                amplitude=fit.params["iA__amplitude"],
            ),
            name="gaussian",
            yaxis="y2",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=fermi_dirac(
                x=y_1d,
                center=fit.params["center"],
                theta=fit.params["theta"],
                amp=fit.params["amp"],
            ),
            name="FD",
            yaxis="y3",
        )
    )

fig.show()

In [ ]:
fit.params

In [ ]:
fit.params["theta"].value / k_B

In [ ]:
tr_functions.sig_to_fwhm(fit.params["iA__sigma"].value)

In [ ]:
## 10.6 K gives 78 meV, 32 K gives 75 meV.
## 32 K and initial condition of 20 meV instead of 80 meV gives a results of 20.1 meV energy resolution and the fit looks good.
## 32 K and initial condition of 30 meV gives a fit of 27.7 meV and the fit looks okay but not great.
## 32 K and initial condition of 10 meV doesn't fit properly (no uncertainties).


### Narrow Limits Around EF ###

## 32 K and initial condition of 20 meV instead of 80 meV gives a results of 17.86 meV energy resolution and the fit looks good.
## 32 K and initial condition of 30 meV gives a fit of 25.27 meV and the fit looks okay but not great.
## 32 K and initial condition of 10 meV doesn't fit properly (no uncertainties).